使用云端qwen-vl模型，完成带文字截图的图，文本的解析转换为文本。

In [4]:
from openai import OpenAI
import os

# 初始化OpenAI客户端
client = OpenAI(
    api_key = "sk-a4402f15d38c41d49ab9d3239988bdd9",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)
try:
    # 测试调用
    response = client.chat.completions.create(
        model="wen-vl-ocr-2025-08-28",  # 使用阿里云的模型
        messages=[
            {"role": "user", "content": "你好"}
        ],
        stream=False
    )
    print(response.choices[0].message.content)
except Exception as e:
    print(f"错误信息: {e}")
reasoning_content = ""  # 定义完整思考过程
answer_content = ""     # 定义完整回复
is_answering = False   # 判断是否结束思考过程并开始回复
enable_thinking = False
# 创建聊天完成请求
completion = client.chat.completions.create(
    model="qwen3-vl-plus",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://www.xunjieshipin.com/wp-content/uploads/2024/04/Dingtalk_20240417144527_%E5%89%AF%E6%9C%AC.jpg"
                    },
                },
                {"type": "text", "text": "请仅输出图像中的文本内容"},
            ],
        },
    ],
    stream=True,
    # enable_thinking 参数开启思考过程，thinking_budget 参数设置最大推理过程 Token 数
    extra_body={
        'enable_thinking': True,
        "thinking_budget": 81920},

    # 解除以下注释会在最后一个chunk返回Token使用量
    stream_options={
        "include_usage": True
    }
)

if enable_thinking:
    print("\n" + "=" * 20 + "思考过程" + "=" * 20 + "\n")

for chunk in completion:
    # 如果chunk.choices为空，则打印usage
    if not chunk.choices:
        print("\nUsage:")
        print(chunk.usage)
    else:
        delta = chunk.choices[0].delta
        # 打印思考过程
        if hasattr(delta, 'reasoning_content') and delta.reasoning_content != None:
            print(delta.reasoning_content, end='', flush=True)
            reasoning_content += delta.reasoning_content
        else:
            # 开始回复
            if delta.content != "" and is_answering is False:
                print("\n" + "=" * 20 + "完整回复" + "=" * 20 + "\n")
                is_answering = True
            # 打印回复过程
            print(delta.content, end='', flush=True)
            answer_content += delta.content

print("=" * 20 + "完整思考过程" + "=" * 20 + "\n")
print(reasoning_content)
print("=" * 20 + "完整回复" + "=" * 20 + "\n")
print(answer_content)

错误信息: Error code: 404 - {'error': {'message': 'The model `wen-vl-ocr-2025-08-28` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}, 'request_id': '79920918-904e-43fd-b0e3-78db6a52eecb'}
用户要求我仅输出图像中的文本内容。首先，我需要仔细查看提供的截图描述，找出其中的文本部分。

首先，界面顶部有“一键识别王”和“快速截图识别”等标签。然后左侧区域有一段手写的文字，需要提取出来。右侧是“识别结果”区域，显示“暂无识别结果”和“立即识别”按钮。

用户特别指出“仅输出图像中的文本内容”，所以需要排除界面元素如按钮名称、菜单项等，只提取图像中显示的文本，也就是左侧手写部分的内容。

查看描述中的左侧区域：“在我们心中的蓝图上，人生之路仿佛是由一条条变得何画出来的，而失去则是必须涂抹掉的笔误。总之，不管失去是一种多么频繁的现象，我们对它反正不习惯。”

需要确认是否有其他文本，比如右侧的“暂无识别结果”是否算图像中的文本？但用户可能指的是被识别的图像内容，即左侧的手写文字，因为右侧是识别结果，可能不算“图像中的文本”，而是软件界面的文本。但用户说“图像中的文本内容”，可能包括整个截图中的所有文本，但需要仔细看要求。

不过用户示例中的问题描述里，截图左侧是手写文字，右侧是识别结果区域。通常“图像中的文本”指的是被识别的图片里的文字，即左侧的手写部分。右侧的“暂无识别结果”是软件界面的提示，可能不算“图像中的文本”，而是软件本身的文字。

再仔细看用户的问题：“请仅输出图像中的文本内容”，这里的“图像”应该指的是左侧被识别的图片内容，即手写的那段文字。因为右侧是识别结果，不是被识别的图像内容。

所以需要提取左侧手写文字部分：

“在我们心中的蓝图上，人生之路仿佛是由一条条变得何画出来的，而失去则是必须涂抹掉的笔误。总之，不管失去是一种多么频繁的现象，我们对它反正不习惯。”

检查是否有错别字，比如“变得何画”可能是“被画”之类的，但按原文输出。